# 5.4 Concatenar

O *faceting* cria gráficos de [múltiplos pequenos](https://en.wikipedia.org/wiki/Small_multiple) que mostram subdivisões separadas dos dados. No entanto, podemos querer criar uma exibição de múltiplas visões com diferentes visões do *mesmo* conjunto de dados (não subconjuntos) ou visões envolvendo *diferentes* conjuntos de dados.

O Altair fornece operadores de *concatenação* para combinar gráficos arbitrários em um gráfico composto. O operador `hconcat` (abreviação `|`) realiza a concatenação horizontal, enquanto o operador `vconcat` (abreviação `&`) realiza a concatenação vertical.



Vamos começar com um gráfico de linha básico mostrando a temperatura máxima média por mês para Nova York e Seattle, assim como vimos antes:


In [ ]:
alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(temp_max):Q'),
  color='local:N'
)

_E o que acontece se quisermos comparar não apenas a temperatura ao longo do tempo, mas também a precipitação e os níveis de vento?_

Vamos criar um gráfico concatenado consistindo de três gráficos. Começaremos definindo uma definição de gráfico "base" que contém todos os aspectos que devem ser compartilhados pelos nossos três gráficos. Podemos então modificar este gráfico base para criar variantes personalizadas, com diferentes codificações do eixo y para os campos `temp_max`, `precipitation` e `wind`. Podemos então concatená-los usando o operador abreviado de pipe (`|`):


In [ ]:
base = alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T', title=None),
  color='location:N'
).properties(
  width=240,
  height=180
)

temp = base.encode(alt.Y('average(temp_max):Q'))
precip = base.encode(alt.Y('average(precipitation):Q'))
wind = base.encode(alt.Y('average(wind):Q'))

temp | precip | wind

Alternativamente, poderíamos usar o método mais explícito `alt.hconcat()` em vez do operador pipe `|`. _Tente reescrever o código acima para usar `hconcat` em vez disso._

A concatenação vertical funciona de maneira semelhante à concatenação horizontal. _Usando o operador `&` (ou o método `alt.vconcat`), modifique o código para usar uma ordenação vertical em vez de uma ordenação horizontal._

Por fim, note que concatenação horizontal e vertical podem ser combinadas. _O que acontece se você escrever algo como `(temp | precip) & wind`?_

_À parte_: Note a importância desses parênteses... o que acontece se você removê-los? Tenha em mente que esses operadores sobrecarregados ainda estão sujeitos às [regras de precedência de operadores do Python](https://docs.python.org/3/reference/expressions.html#operator-precedence), e, portanto, a concatenação vertical com `&` terá precedência sobre a concatenação horizontal com `|`!

Como vamos revisar mais tarde, os operadores de concatenação permitem combinar qualquer e todos os gráficos em um painel de múltiplas visões!


## 5.5 Repetir


Os operadores de concatenação acima são bastante gerais, permitindo que gráficos arbitrários sejam compostos. No entanto, o exemplo acima ainda era um pouco verboso: temos três gráficos muito semelhantes, mas temos que defini-los separadamente e depois concatená-los.

Para casos onde apenas uma ou duas variáveis estão mudando, o operador `repeat` oferece um atalho conveniente para criar múltiplos gráficos. Dada uma especificação de *template* com algumas variáveis livres, o operador repeat então criará um gráfico para cada atribuição especificada para essas variáveis.

Vamos recriar nosso exemplo de concatenação acima usando o operador `repeat`. O único aspecto que muda entre os gráficos é a escolha do campo de dados para o canal de codificação `y`. Para criar uma especificação de template, podemos usar a *variável repetidora* `alt.repeat('column')` como nosso campo do eixo y. Esse código simplesmente declara que queremos usar a variável atribuída ao repetidor `column`, que organiza gráficos repetidos em uma direção horizontal. (Como o repetidor fornece apenas o nome do campo, precisamos especificar separadamente o tipo de dados do campo como `type='quantitative'`.)

Em seguida, invocamos o método `repeat`, passando os nomes dos campos de dados para cada coluna:


In [ ]:
alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T',title=None),
  alt.Y(alt.repeat('column'), aggregate='average', type='quantitative'),
  color='location:N'
).properties(
  width=240,
  height=180
).repeat(
  column=['temp_max', 'precipitation', 'wind']
)

A repetição é suportada tanto para colunas quanto para linhas. _O que acontece se você modificar o código acima para usar `row` em vez de `column`?_

Também podemos usar a repetição de `row` e `column` juntas! Uma visualização comum para análise exploratória de dados é a [matriz de gráficos de dispersão (ou SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices). Dada uma coleção de variáveis a serem analisadas, um SPLOM fornece uma grade de todos os gráficos pareados dessas variáveis, permitindo-nos avaliar associações potenciais.

Vamos usar o operador `repeat` para criar um SPLOM para os campos `temp_max`, `precipitation` e `wind`. Primeiro, criamos nossa especificação de template, com variáveis de repetição para os campos de dados dos eixos x e y. Em seguida, invocamos `repeat`, passando arrays de nomes de campos para usar tanto para `row` quanto para `column`. O Altair então gerará o [produto cruzado (ou, produto cartesiano)](https://en.wikipedia.org/wiki/Cartesian_product) para criar o espaço completo de gráficos repetidos:


In [ ]:
alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=150,
  height=150
).repeat(
  data=weather,
  row=['temp_max', 'precipitation', 'wind'],
  column=['wind', 'precipitation', 'temp_max']
).transform_filter(
  'datum.location == "Seattle"'
)

_Olhando para esses gráficos, não parece haver uma associação forte entre precipitação e vento, embora vejamos que eventos extremos de vento e precipitação ocorrem em faixas de temperatura semelhantes (~5-15° C). No entanto, essa observação não é particularmente surpreendente: se revisarmos nosso histograma no início da seção de facetamento, podemos ver claramente que os dias com temperaturas máximas na faixa de 5-15° C são os que mais ocorrem._

*Modifique o código acima para obter uma melhor compreensão da repetição de gráficos. Tente adicionar outra variável (`temp_min`) ao SPLOM. O que acontece se você reorganizar a ordem dos nomes dos campos nos parâmetros `row` ou `column` para o operador `repeat`?*

_Por fim, para realmente apreciar o que o operador `repeat` oferece, reserve um momento para imaginar como você recriaria o SPLOM acima usando apenas `hconcat` e `vconcat`!_
